## **Imports**

In [34]:
%matplotlib inline
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import cv2
import torch   
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import lr_scheduler
import time
import copy

In [35]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Running on the GPU")
else: 
  device = torch.device("cpu")
  print("Running on the CPU")

Running on the GPU


In [36]:
# declate image size
imsize = 105

In [37]:
# Mount to Google Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [39]:
! ls '/content/drive/My Drive/Mote_Manatee'

jul23.pth  MMLDUs_BatchA  out3.npy  out_jul23.npy


In [40]:
# Read the npy file
manatee_data = np.load('/content/drive/My Drive/Mote_Manatee/out_jul23.npy', allow_pickle=True)

In [41]:
manatee_data.shape

(1659, 3)

## Separate training and validation data sets

In [42]:
VAL_PCT = 0.1 # reserve 10% for validation
val_size = int(manatee_data.shape[0]*VAL_PCT)
print(val_size)

165


In [43]:
training_data = manatee_data[:-val_size]
validation_data = manatee_data[-val_size:]

# **Dataset class loader**

In [44]:
class ManateeDataset(Dataset):
  """ Manatee Dataset Class """

  def __init__(self, manatee_data, imsize, transform=None):
    """
    Args: 
        manatee_data (np array): numpy structured as (x1,x2,y)
        transform (optional): optional transform to be applied on a sample
    """
    self.manatee_data = manatee_data
    self.transform = transform
    self.imsize = imsize

  def __len__(self):
    return self.manatee_data.shape[0]

  def __getitem__(self, idx):
    x1 = torch.Tensor(self.manatee_data[idx][0]).view(-1, self.imsize, self.imsize)
    x2 = torch.Tensor(self.manatee_data[idx][1]).view(-1, self.imsize, self.imsize)
    y = torch.Tensor(self.manatee_data[idx][2])
    return x1, x2, y

In [45]:
train_data = ManateeDataset(training_data, 105)
val_data = ManateeDataset(validation_data, 105)

# **Neural Net Definition**

In [46]:
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 10),  # 64@96*96
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 64@48*48
            nn.Conv2d(64, 128, 7),
            nn.ReLU(),    # 128@42*42
            nn.MaxPool2d(2),   # 128@21*21
            nn.Conv2d(128, 128, 4),
            nn.ReLU(), # 128@18*18
            nn.MaxPool2d(2), # 128@9*9
            nn.Conv2d(128, 256, 4),
            nn.ReLU(),   # 256@6*6
        )
        self.liner = nn.Sequential(nn.Linear(9216, 4096), nn.Sigmoid())
        self.out = nn.Linear(4096, 2)

    def forward_both(self, x1, x2):
      out1 = self.forward_one(x1)
      out2 = self.forward_one(x2)
      #euclidean_distance = F.pairwise_distance(out1, out2, keepdim = True)
      cos = nn.CosineSimilarity()
      output = cos(out1,out2)
      return output

    def forward_one(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        output = self.liner(x)
        return output

    def forward(self, input1, input2):
        out1 = self.forward_one(input1)
        out2 = self.forward_one(input2)
        return out1, out2

In [47]:
net = Siamese()
# push to device
#net = net.to(device)

# **Loss Function Definition**

In [48]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

## **Transform Data to PyTorch Tensors**

In [ ]:
"""
X2 = []
for labeled_pair in range(training_data.shape[0]):
  img1 = np.array(training_data[labeled_pair][1])
  # append
  X2.append(img1)
"""

In [22]:
#X1 = torch.Tensor([training_data[labeled_pair][0] for labeled_pair in range(training_data.shape[0] )]).view(-1, imsize, imsize)
#X2 = torch.Tensor([training_data[labeled_pair][1] for labeled_pair in range(training_data.shape[0] )]).view(-1, imsize, imsize)
#y = torch.Tensor([training_data[labeled_pair][2] for labeled_pair in range(training_data.shape[0] )])

#print(X1.shape)
#print(X2.shape)
#print(y.shape)

# **Train**

In [49]:
#Create DataLoader iterators
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=2)

In [50]:
dataloaders = {'train': train_loader, 'val':val_loader}
dataset_sizes = {'train': training_data.shape[0], 'val': validation_data.shape[0]}

In [55]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for x1,x2,y in dataloaders[phase]:
                #x1 = x1.to(device)
                #x2 = x2.to(device)
                #y = y.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    output1, output2 = model(x1,x2)
                    # predict similarity 
                    preds = model.forward_both(x1,x2)
                    # constrain (0-1)
                    preds = torch.round(preds)
                    # compute loss
                    loss = criterion(output1, output2, y)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * x1.size(0)
                running_corrects += torch.sum(preds == torch.max(y))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [56]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = ContrastiveLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [57]:
# train and evaluate
model = train_model(net, loss_function, optimizer, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


KeyboardInterrupt: ignored

## **Test Trained Network**

In [32]:
torch.save(model.state_dict, '/content/drive/My Drive/Mote_Manatee/jul23.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Siamese. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [33]:
def oneshot(model, img1,img2, imsize=105):
  # Gives the feature vector of both inputs
  output1, output2 = model(img1.view(-1,1,imsize,imsize),img2.view(-1,1,imsize,imsize))
  # Compute the distance
  euclidean_distance = F.pairwise_distance(output1, output2)
  print(euclidean_distance)
